In [3]:
import pandas as pd
import numpy as np
import datetime
import random

# ------------------ MASTER INTENSITY PROFILE (Simplified) ------------------
def generate_master_intensity_profile():
    """Returns an empty list as the master profile is no longer used."""
    return []

# ------------------ STORM CELL GENERATION (Modified) ------------------
def generate_storm_cell_lifecycle(
    cell_id,
    start_date,
    end_date,
    target_peak_intensity_dbz=75,
    time_step_minutes=5,
    base_size_pixels=120,
    base_vx=0.0,
    base_vy=2.0,
    movement_randomness_scale=0.2
):
    """
    Generates a single storm cell's lifecycle with all requested properties.
    The intensity calculation is fixed to ensure a consistent peak intensity.
    """
    records = []

    # Random formation time
    time_range_seconds = (end_date - start_date).total_seconds()
    formation_time = start_date + datetime.timedelta(seconds=random.uniform(0, time_range_seconds))

    # Lifetime from Normal Distribution with min/max bounds
    mean_lifetime_hours = 40 / 60  # ~40 minutes
    std_dev_lifetime_hours = 0.3
    min_lifetime_hours = 15 / 60   # 15 minutes
    max_lifetime_hours = 1.5       # 1.5 hours
    
    lifetime_hours = np.random.normal(mean_lifetime_hours, std_dev_lifetime_hours)
    lifetime_hours = max(min_lifetime_hours, min(max_lifetime_hours, lifetime_hours))

    lifetime_delta = datetime.timedelta(hours=lifetime_hours)
    dissipation_time = formation_time + lifetime_delta

    # Starting position
    x_position, y_position = 0, 0

    previous_intensity = 0
    peak_relative_time = 0.5 

    current_time = formation_time
    while current_time <= dissipation_time:
        time_elapsed_seconds = (current_time - formation_time).total_seconds()
        
        # Normalize time elapsed to a 0-1 range
        progress = time_elapsed_seconds / lifetime_delta.total_seconds()
        
        # --- FIX: Ensure peak intensity is hit exactly at the midpoint ---
        if abs(progress - peak_relative_time) < (time_step_minutes / 60) / (lifetime_hours * 2):
            intensity_dbz = target_peak_intensity_dbz
        elif progress < peak_relative_time:
            # Growth phase: Logistic curve from 0 to target_peak_intensity_dbz
            intensity_dbz = target_peak_intensity_dbz / (1 + np.exp(-10 * (progress - peak_relative_time)))
        else:
            # Decay phase: Exponential decay from target_peak_intensity_dbz to 0
            decay_progress = (progress - peak_relative_time) / (1 - peak_relative_time)
            intensity_dbz = target_peak_intensity_dbz * np.exp(-10 * decay_progress)

        # Ensure intensity does not go below zero
        intensity_dbz = max(0.0, intensity_dbz)
        
        # Intensity change rate
        intensity_change_rate = (intensity_dbz - previous_intensity) / (time_step_minutes / 60)

        # Size scaling
        size_pixels = 20 + (intensity_dbz / target_peak_intensity_dbz) * base_size_pixels if target_peak_intensity_dbz > 0 else 20
        size_pixels = size_pixels * random.uniform(0.9, 1.1)  # ±10% size noise
        size_pixels = max(0, int(size_pixels))

        # Rainfall calculation with multiplicative noise (±10%)
        base_rainfall = 0.08 * (intensity_dbz ** 1.5)
        rainfall_factor = 1.0 + random.uniform(-0.1, 0.1)  # 10% up or down
        rainfall_mmhr = base_rainfall * rainfall_factor
        rainfall_mmhr = max(0.0, rainfall_mmhr)

        if intensity_dbz == 0.0:
            rainfall_mmhr = 0.0

        # Movement (unchanged)
        vx = base_vx + np.random.normal(0, movement_randomness_scale)
        vy = base_vy + np.random.normal(0, movement_randomness_scale)
        x_position += vx * (time_step_minutes / 60)
        y_position += vy * (time_step_minutes / 60)

        records.append({
            'cell_id': cell_id,
            'timestamp_utc': current_time,
            'formation_time_utc': formation_time,
            'dissipation_time_utc': dissipation_time,
            'lifetime_hours': lifetime_hours,
            'time_since_formation_hours': time_elapsed_seconds / 3600,
            'x_position': x_position,
            'y_position': y_position,
            'size_pixels': size_pixels,
            'intensity_dbz': intensity_dbz,
            'rainfall_mm_per_hr': rainfall_mmhr,
            'intensity_change_rate': intensity_change_rate
        })

        previous_intensity = intensity_dbz
        current_time += datetime.timedelta(minutes=time_step_minutes)

    return records


# ------------------ RUN SIMULATION ------------------
print("--- Generating storms with all specified changes ---")

all_storm_data = []
num_simulated_cells = 50000
overall_start_date = datetime.datetime(2024, 8, 1, 0, 0, 0)
overall_end_date = datetime.datetime(2024, 8, 5, 23, 59, 59)

master_intensity_profile = generate_master_intensity_profile()

for i in range(num_simulated_cells):
    cell_records = generate_storm_cell_lifecycle(
        cell_id=f'StormCell_{i+1:05d}',
        start_date=overall_start_date,
        end_date=overall_end_date,
        target_peak_intensity_dbz=75,
        base_vx=0.0,
        base_vy=2.0,
        movement_randomness_scale=0.2
    )
    all_storm_data.extend(cell_records)

# Create DataFrame
storm_df = pd.DataFrame(all_storm_data)
storm_df = storm_df.sort_values(by=['cell_id', 'timestamp_utc']).reset_index(drop=True)

# Save CSV
output_file = "scenario6_fixed_peak_50000.csv"
storm_df.to_csv(output_file, index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

print(f"Generated {len(storm_df)} rows for {num_simulated_cells} storms → saved to {output_file}.")


--- Generating storms with all specified changes ---
Generated 435202 rows for 50000 storms → saved to scenario6_fixed_peak_50000.csv.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------------------- Load and Inspect Data --------------------
# This script assumes the CSV file is in the same directory.
# If you get a FileNotFoundError, please check the file path.
file_path = "scenario6_fixed_peak_50000.csv"
try:
    storm_df = pd.read_csv(file_path)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure the file is in the same directory as this script.")
    # Exit the script gracefully if the file is not found
    exit()

# Convert relevant columns to appropriate data types
storm_df['timestamp_utc'] = pd.to_datetime(storm_df['timestamp_utc'])
storm_df['formation_time_utc'] = pd.to_datetime(storm_df['formation_time_utc'])
storm_df['dissipation_time_utc'] = pd.to_datetime(storm_df['dissipation_time_utc'])

# Display initial information about the DataFrame
print("\nDataFrame Info:")
print(storm_df.info())

print("\nFirst 5 rows of the DataFrame:")
print(storm_df.head())

# -------------------- 1. Verify Peak Intensity --------------------
print("\n--- Verifying Peak Intensity ---")
# Group by storm cell and find the maximum intensity for each
peak_intensities = storm_df.groupby('cell_id')['intensity_dbz'].max()

# Check if all peak intensities are approximately the same
min_peak = peak_intensities.min()
max_peak = peak_intensities.max()
expected_peak = 75.0

is_same_peak_intensity = np.isclose(min_peak, max_peak, atol=0.1)
print(f"Minimum peak intensity found: {min_peak:.2f} dBZ")
print(f"Maximum peak intensity found: {max_peak:.2f} dBZ")
print(f"Are all peak intensities approximately the same (within a tolerance of 0.1)? {'Yes' if is_same_peak_intensity else 'No'}")
print(f"Are all peak intensities approximately equal to the target of {expected_peak} dBZ? {np.all(np.isclose(peak_intensities, expected_peak, atol=0.5))}")


# -------------------- 2. & 3. Visualize Two Unique Storm Lifecycles --------------------
print("\n--- Visualizing Two Unique Storm Lifecycles ---")

# Get two unique storm IDs with different lifetimes
all_lifetimes = storm_df.groupby('cell_id')['lifetime_hours'].first()
short_storm_id = all_lifetimes.idxmin()
long_storm_id = all_lifetimes.idxmax()

# Filter the data for these two storms
storm1_df = storm_df[storm_df['cell_id'] == short_storm_id]
storm2_df = storm_df[storm_df['cell_id'] == long_storm_id]

# Create a figure with a 2x1 grid of subplots
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=False)
fig.suptitle('Comparison of Two Storm Cell Lifecycles', fontsize=16)

# Plot Intensity vs. Lifetime
axes[0].plot(storm1_df['time_since_formation_hours'], storm1_df['intensity_dbz'], label=f'Storm ID: {short_storm_id} (Lifetime: {storm1_df["lifetime_hours"].iloc[0]:.2f} hrs)')
axes[0].plot(storm2_df['time_since_formation_hours'], storm2_df['intensity_dbz'], label=f'Storm ID: {long_storm_id} (Lifetime: {storm2_df["lifetime_hours"].iloc[0]:.2f} hrs)')
axes[0].set_title('Intensity vs. Time Since Formation')
axes[0].set_xlabel('Time Since Formation (hours)')
axes[0].set_ylabel('Intensity (dBZ)')
axes[0].legend()
axes[0].grid(True)

# Plot Rainfall vs. Lifetime
axes[1].plot(storm1_df['time_since_formation_hours'], storm1_df['rainfall_mm_per_hr'], label=f'Storm ID: {short_storm_id}')
axes[1].plot(storm2_df['time_since_formation_hours'], storm2_df['rainfall_mm_per_hr'], label=f'Storm ID: {long_storm_id}')
axes[1].set_title('Rainfall vs. Time Since Formation')
axes[1].set_xlabel('Time Since Formation (hours)')
axes[1].set_ylabel('Rainfall (mm/hr)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('storm_lifecycles.png')
plt.close()
print("Plots saved to 'storm_lifecycles.png'.")

# -------------------- 4. Calculate Statistics for All Storms --------------------
print("\n--- Calculating Statistics for All Storms ---")

# Group by storm cell ID to calculate per-storm statistics
grouped_storms = storm_df.groupby('cell_id')

# Calculate key metrics for each storm
storm_stats_df = pd.DataFrame({
    'lifetime_hours': grouped_storms['lifetime_hours'].first(),
    'peak_intensity_dbz': grouped_storms['intensity_dbz'].max(),
    'peak_rainfall_mmhr': grouped_storms['rainfall_mm_per_hr'].max(),
    'average_intensity_dbz': grouped_storms['intensity_dbz'].mean(),
    'total_rainfall_mm': grouped_storms['rainfall_mm_per_hr'].sum() * (5/60) # Sum of rate * time_step_in_hours
})

# Calculate descriptive statistics for the entire simulation
overall_stats_description = storm_stats_df.describe()

print("\nDescriptive Statistics for all 50,000 Storms:")
print(overall_stats_description)

Data loaded successfully.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435202 entries, 0 to 435201
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   cell_id                     435202 non-null  object        
 1   timestamp_utc               435202 non-null  datetime64[ns]
 2   formation_time_utc          435202 non-null  datetime64[ns]
 3   dissipation_time_utc        435202 non-null  datetime64[ns]
 4   lifetime_hours              435202 non-null  float64       
 5   time_since_formation_hours  435202 non-null  float64       
 6   x_position                  435202 non-null  float64       
 7   y_position                  435202 non-null  float64       
 8   size_pixels                 435202 non-null  int64         
 9   intensity_dbz               435202 non-null  float64       
 10  rainfall_mm_per_hr          435202 non-null  float64       
 